# FF++ Staged Fine-tuning (FAST Version)

**Key optimization**: Pre-extract all faces BEFORE training.

| Phase | Time |
|-------|------|
| Pre-extraction | ~30-45 min (one-time) |
| Stage A (2 epochs) | ~30 min |
| Stage B (8 epochs) | ~2 hrs |
| **Total** | **~3-4 hours** |

Compare to on-the-fly extraction: ~19 hours

In [ ]:
# 1. GPU Check
import torch
if not torch.cuda.is_available():
    raise SystemExit("GPU required! Go to Runtime > Change runtime type > GPU")
print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# 2. Mount Drive & Clone Repo
from google.colab import drive
drive.mount('/content/drive')

!rm -rf /content/Team-Converge
!git clone https://github.com/Incharajayaram/Team-Converge.git /content/Team-Converge
%cd /content/Team-Converge/Finetune1

In [ ]:
# 3. Install Dependencies
!pip install -q mediapipe pyyaml tqdm gdown scikit-learn

In [ ]:
# 4. Download FF++ Data (17GB)
import os

# Option 1: Copy from user's Drive (Reliable "Trick")
# This bypasses the "Too many users have viewed..." error
if os.path.exists('/content/drive/MyDrive/ffpp_data_new.zip'):
    print("Copying from Drive (Fast & Reliable)...")
    !cp '/content/drive/MyDrive/ffpp_data_new.zip' /content/ffpp_data.zip
elif os.path.exists('/content/drive/MyDrive/ffpp_data.zip'):
    print("Copying from Drive (Fast & Reliable)...")
    !cp '/content/drive/MyDrive/ffpp_data.zip' /content/ffpp_data.zip
else: 
    # Option 2: Download from link (Prone to rate limits)
    print("File not found in Drive. Attempting download (may fail with rate limit)...")
    FILE_ID = "1a7X9Cjv3gsj4qC6kcDq6VLNl7eR3osoy"
    ZIP_PATH = "/content/ffpp_data.zip"
    !pip install -q --upgrade gdown
    !gdown --id {FILE_ID} --output {ZIP_PATH} --fuzzy

ZIP_PATH = "/content/ffpp_data.zip"
if os.path.exists(ZIP_PATH):
    size_gb = os.path.getsize(ZIP_PATH) / 1e9
    print(f"File size: {size_gb:.2f} GB")
    if size_gb < 15:
        raise ValueError(f"Download incomplete! Only {size_gb:.2f} GB. Try copying manually to Drive first.")
else:
    raise FileNotFoundError("ffpp_data.zip not found! Upload 'ffpp_data_new.zip' to your Drive root.")

In [ ]:
# 5. Extract Data
!rm -rf /content/data/raw/ffpp
!mkdir -p /content/data/raw/ffpp
!unzip -q /content/ffpp_data.zip -d /content/data/raw/ffpp

import os
FFPP_ROOT = "/content/data/raw/ffpp/FaceForensics++_C23"
if not os.path.exists(FFPP_ROOT):
    FFPP_ROOT = "/content/data/raw/ffpp"
os.environ['FFPP_ROOT'] = FFPP_ROOT
print(f"FFPP_ROOT: {FFPP_ROOT}")

In [ ]:
# 6. Generate Video Index
import os
os.chdir('/content/Team-Converge/Finetune1')

from pathlib import Path
from utils.indexing import build_master_index

FFPP_ROOT = os.environ.get('FFPP_ROOT')
output_csv = Path("data/index/videos_master.csv")
output_csv.parent.mkdir(parents=True, exist_ok=True)

videos = build_master_index(ffpp_root=Path(FFPP_ROOT), output_path=output_csv)
print(f"Indexed {len(videos)} videos")

In [ ]:
# 7. Generate Fast Manifests (no ffprobe)
import csv
from pathlib import Path
from utils.indexing import load_master_index
from utils.face_extraction import generate_sample_id

videos = load_master_index(Path("data/index/videos_master.csv"))

samples = []
for v in videos:
    k = 10 if v.split != 'test' else 20
    for i in range(k):
        ts = 0.5 + i * 0.8
        samples.append({
            'sample_id': generate_sample_id(),
            'dataset': v.dataset, 'split': v.split, 'label': v.label,
            'method': v.method, 'group_id': v.group_id,
            'video_id': v.video_id, 'video_path': v.video_path,
            'timestamp': ts, 'filepath': ''
        })

out_dir = Path('artifacts/manifests')
out_dir.mkdir(parents=True, exist_ok=True)

for split in ['train', 'val', 'test']:
    split_samples = [s for s in samples if s['split'] == split]
    with open(out_dir / f'{split}.csv', 'w', newline='') as f:
        w = csv.DictWriter(f, fieldnames=list(samples[0].keys()))
        w.writeheader()
        w.writerows(split_samples)
    print(f"{split}: {len(split_samples)} samples")

In [ ]:
# 8. PRE-EXTRACT ALL FACES (30-45 min, CRITICAL for fast training!)
import os
FFPP_ROOT = os.environ.get('FFPP_ROOT', '/content/data/raw/ffpp/FaceForensics++_C23')

print("="*60)
print("PRE-EXTRACTING FACES (this takes 30-45 min)")
print("This is a ONE-TIME cost that makes training 10x faster!")
print("="*60)

# Extract train faces
!python preextract_faces.py \
    --ffpp_root {FFPP_ROOT} \
    --manifest artifacts/manifests/train.csv \
    --cache_dir /content/cache/faces \
    --workers 8

# Extract val faces
!python preextract_faces.py \
    --ffpp_root {FFPP_ROOT} \
    --manifest artifacts/manifests/val.csv \
    --cache_dir /content/cache/faces \
    --workers 8

print("\nPre-extraction complete! Training will be FAST now.")

In [ ]:
# 9. Create output directories
!mkdir -p /content/drive/MyDrive/ffpp_training/staged
!mkdir -p /content/cache/faces
print("Output dir: /content/drive/MyDrive/ffpp_training/staged")

In [ ]:
# 10. Stage A: Head-only training (2 epochs) - NOW FAST!
import os
FFPP_ROOT = os.environ.get('FFPP_ROOT', '/content/data/raw/ffpp/FaceForensics++_C23')

print("="*60)
print("STAGE A: Head-only training (2 epochs)")
print("Expected time: ~30 min (faces are pre-extracted!)")
print("="*60)

!python train_staged.py --config config.yaml \
    --override dataset.ffpp_root={FFPP_ROOT} \
    --override caching.cache_dir=/content/cache/faces \
    --stages A \
    --output_dir /content/drive/MyDrive/ffpp_training/staged

In [ ]:
# 11. Stage B: Partial unfreeze (8 epochs)
import os
FFPP_ROOT = os.environ.get('FFPP_ROOT', '/content/data/raw/ffpp/FaceForensics++_C23')

checkpoint = "/content/drive/MyDrive/ffpp_training/staged/best_model.pt"
if not os.path.exists(checkpoint):
    raise FileNotFoundError(f"Stage A checkpoint not found: {checkpoint}")

print("="*60)
print("STAGE B: Partial unfreeze - layer4 (8 epochs)")
print("Expected time: ~2 hrs")
print("="*60)

!python train_staged.py --config config.yaml \
    --override dataset.ffpp_root={FFPP_ROOT} \
    --override caching.cache_dir=/content/cache/faces \
    --stages B \
    --resume {checkpoint} \
    --output_dir /content/drive/MyDrive/ffpp_training/staged

In [ ]:
# 12. View Training History
import json
import matplotlib.pyplot as plt
import os

history_path = '/content/drive/MyDrive/ffpp_training/staged/training_history.json'
if os.path.exists(history_path):
    with open(history_path) as f:
        history = json.load(f)
    
    epochs = [h['epoch'] for h in history]
    train_loss = [h['train_loss'] for h in history]
    val_loss = [h['val_loss'] for h in history]
    val_auc = [h.get('val_auc', 0.5) for h in history]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    ax1.plot(epochs, train_loss, 'b-', label='Train')
    ax1.plot(epochs, val_loss, 'r-', label='Val')
    ax1.set_xlabel('Epoch'); ax1.set_ylabel('Loss'); ax1.legend(); ax1.grid(True)
    ax2.plot(epochs, val_auc, 'g-'); ax2.set_ylim(0.5, 1.0)
    ax2.set_xlabel('Epoch'); ax2.set_ylabel('AUC'); ax2.grid(True)
    plt.tight_layout(); plt.show()
    
    print(f"Best val_loss: {min(val_loss):.4f}")
    print(f"Best val_auc: {max(val_auc):.4f}")
else:
    print(f"History not found: {history_path}")

In [ ]:
# 13. Save Final Model
import shutil
import os

src = '/content/drive/MyDrive/ffpp_training/staged/best_model.pt'
dst = '/content/drive/MyDrive/ffpp_training/final_model.pt'

if os.path.exists(src):
    shutil.copy(src, dst)
    print(f"Saved: {dst}")
    print(f"Size: {os.path.getsize(dst)/1e6:.1f} MB")
else:
    print(f"Model not found: {src}")

print("\n" + "="*60)
print("TRAINING COMPLETE!")
print("="*60)